In [1]:
#모든 행, 열 보기
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 함수정의

In [2]:
# 데이터프레임 생성
def teenager_preprocessing(df_name, sheet_list):
    result_list = []
    for sheet_num in sheet_list:
        df = pd.read_excel(f'./data/청소년건강형태조사/{df_name}.xlsx', sheet_name=sheet_num)
        # 전국 시도별 구분 행만 가져오기
        df_1 = df.iloc[15:33, -3:]
        df_1['시도'] = df.iloc[15:33, 2]
        df_1 = df_1.drop(df_1.index[0]).set_index('시도')
        df_1.columns = ['표본 수', '분율', '표준오차']
        result_list.append(df_1)
        
    result = pd.concat(result_list, axis=1)
    return result


# 멀티컬럼 생성
def create_multi_column(df_list):
    multi_col_df_list = []
    for i, df in enumerate(df_list):
        # [표본 수, 분율, 표준오차] 마다 T1, T2, ... Tn 으로 멀티컬럼 구성
        num_col = len(df.columns) // 3
        # 컬럼개수가 9개라면 [T1,T1,T1,T2,T2,T2,T3,T3,T3] 로 만들어서 저장
        multi_col = [f'T{j + 1}' for j in range(num_col) for _ in range(3)]
        multi_col_df = pd.DataFrame(df.values, index=df.index, columns=[multi_col, df.columns])
        multi_col_df_list.append(multi_col_df)
    return multi_col_df_list


# 순위부여 함수 (감・가점 적용 전)
def give_rank_to_df(df_list):
    result_df_list = []

    for idx, multiCol_df in enumerate(df_list):
        # 순위부여
        result = multiCol_df.iloc[:, 1::3].rank(axis=0).sum(axis=1).rank(ascending=False).astype(int)
        # 결과를 데이터프레임으로 변환해 리스트에 추가
        result_df_list.append(pd.DataFrame(result, columns=[f'T{idx+1}']))

    # 하나의 데이터프레임으로 만들기
    final_result_df = pd.concat(result_df_list, axis=1)
    return final_result_df


# 순위부여 함수 (감・가점 적용 후)
def give_rank_to_df_2(df_list, col_list):
    result_df_list = []

    for idx, multiCol_df in enumerate(df_list):
        if idx == 8:  # 정신건강은 합치지 않고 각각의 데이터로 점수 산출
            defalut_df = include_multiCol_df_list[idx].iloc[:, 1::3]

            multi_col_1 = [f'{j + 1}' for j in range(len(defalut_df.columns))]
            defalut_df.columns = multi_col_1
            defalut_df.columns = defalut_df.columns.astype(int)

            plus_df = defalut_df[col_list[idx]].rank(method='min') 
            minus_df = defalut_df.loc[:, ~defalut_df.columns.isin(col_list[idx])].rank(method='min',
                                                                                   ascending=False)
            result = pd.concat([plus_df, minus_df], axis=1)
            result = result.sort_index(axis=1)

            result.columns = ['T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16']
            result = result.astype(int)
            result_df_list.append(pd.DataFrame(result))

        else:  
            defalut_df = multiCol_df.iloc[:, 1::3] # 분율만 인덱싱
            # 가점 열별로 랭크부여 // 낮은 분율부터 1점 ~ 높은 분율 17점 
            plus_df = defalut_df[col_list[idx]].rank(method='min') 
            # 감점 열별로 랭크부여 // 낮은 분율부터 17점 ~ 높은 분율 1점 
            minus_df = defalut_df.loc[:, ~defalut_df.columns.isin(col_list[idx])].rank(method='min',
                                                                                       ascending=False)
            first_result = pd.concat([plus_df, minus_df], axis=1)
            # 시도별 랭크부여
            second_result = first_result.sum(axis=1).rank(method='min').astype(int)

            # 결과를 데이터프레임으로 변환해 리스트에 추가
            result_df_list.append(pd.DataFrame(second_result, columns=[f'T{idx+1}']))

    # 하나의 데이터프레임으로 만들기
    final_result_df = pd.concat(result_df_list, axis=1)
    return final_result_df

### 전처리 함수적용

In [3]:
# 데이터프레임 만들기 함수 적용
activity = teenager_preprocessing('신체활동', [1,2,3])
obesity = teenager_preprocessing('비만', [1,2,6])
smoke = teenager_preprocessing('흡연', [2,4,5,9,11,13,15,17,18]) # 6번 시도 구분 없음(보류)
drinking = teenager_preprocessing('음주', [2,5,7,10])
eating = teenager_preprocessing('식생활', [1,2,3,5])
oral = teenager_preprocessing('구강', [1,2,3,4,5])
hygiene = teenager_preprocessing('개인위생', [1,2,3,4,5])
AtopicAsthma = teenager_preprocessing('아토피천식', [1,4,5]) # 3번 시도 구분 없음(보류)
# addicted = teenager_preprocessing('중독', [1,3,4,7])  # 전부 시도 구분 없음(보류)
mental = teenager_preprocessing('정신건강', [1,3,4,5,7,10,11,12]) # 6번 시도 구분 없음(보류)

In [4]:
df_name_list = [activity, obesity, smoke, drinking, eating, oral, hygiene, AtopicAsthma, mental]

# 멀티컬럼 함수 적용
include_multiCol_df_list = create_multi_column(df_name_list)

In [5]:
# Null 확인
for df in df_name_list:
    print(df.isnull().sum())

표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
dtype: int64
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
dtype: int64
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
dtype: int64
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
dtype: int64
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
dtype: int64
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
dtype: int64
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준오차    0
표본 수    0
분율      0
표준

### 식생활, 개인위생 교육항목 전처리 및 csv 파일로 내보내기

In [6]:
dietary_education = teenager_preprocessing('식생활', [6])
hygiene = teenager_preprocessing('개인위생', [6])

In [7]:
diet = dietary_education['분율']
hygi = hygiene['분율']

personal_edu = pd.concat([diet, hygi], axis=1)
personal_edu.to_csv('personal_edu.csv')

# 함수적용 결과 및 컬럼 설명 (총 9개)

### include_multiCol_df_list[0] : [신체] 신체활동
    - T1 : "최근 7일 동안 운동 종류 상관없이 심장박동이 평상시보다 증가하거나, 숨이 찬 정도의 신체활동을 하루에 총합이 60분 이상 한 날이 5일 이상인 사람의 분율"
    - T2 : "최근 7일 동안 고강도 신체활동(조깅(달리기), 축구, 농구, 단식(1:1) 테니스, 태권도, 등산, 빠른 속도의 자전거 타기, 빠른 수영, 줄넘기, 무거운 물건 나르기, 에어로빅 등)을 3일 이상한 사람의 분율"
    - T3 : 최근 7일 동안 근력강화운동(팔굽혀펴기, 윗몸일으키기, 역기 들기, 아령, 철봉, 평행봉 등)을 3일 이상 한 사람의 분율

In [8]:
include_multiCol_df_list[1].iloc[:, 1::3].info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries,   서울 to   제주
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   (T1, 분율)  17 non-null     object
 1   (T2, 분율)  17 non-null     object
 2   (T3, 분율)  17 non-null     object
dtypes: object(3)
memory usage: 544.0+ bytes


In [9]:
include_multiCol_df_list[0].iloc[:, 1::3] # 신체활동

# 1. 항목들을 더하고 퍙균내서 PCA ?
# 2. 항목들을 정규화 시켜서 PCA ?
# 3. 순위로 PCA ?

,T1,T2,T3
,분율,분율,분율
시도,,,
서울,17.7,42.0,24.6
부산,17.1,41.6,26.0
대구,16.1,40.1,25.4
인천,18.7,39.0,24.7
광주,15.8,41.1,26.0
대전,17.6,40.0,24.6
울산,19.1,45.7,26.9
세종,15.3,41.7,24.4


### include_multiCol_df_list[1] : [신체] 비만
    - T1 : 2017년 소아청소년 성장도표 연령별 체질량지수 기준 95백분위수 이상인 사람의 분율
    - T2 : 2017년 소아청소년 성장도표 연령별 체질량지수 기준 85백분위수 이상에서 95백분위수 미만인 사람의 분율
    - T3 : 2017년 소아청소년 성장도표 연령별 체질량지수 기준 85백분위수 미만인 자 중에서 자신의 체형을 살이 찐 편이라고 인지하는 사람의 분율

In [10]:
include_multiCol_df_list[1]

T1                  T2                 T3             
       표본 수    분율   표준오차   표본 수    분율   표준오차  표본 수    분율   표준오차
시도                                                             
  서울   7205  11.0  (0.5)   7205   8.2  (0.4)  5813  22.5  (0.7)
  부산   3081  12.1  (0.7)   3081   9.2  (0.5)  2416  21.8  (1.1)
  대구   2573  12.6  (0.7)   2573   8.6  (0.5)  2029  19.2  (0.8)
  인천   2832  12.6  (0.8)   2832   9.4  (0.7)  2201  22.0  (1.1)
  광주   2049  10.6  (0.6)   2049   8.4  (0.6)  1669  21.4  (1.0)
  대전   1808  11.0  (0.9)   1808   8.5  (0.9)  1452  20.8  (1.1)
  울산   1618  11.3  (1.0)   1618   8.7  (0.6)  1279  22.1  (1.2)
  세종   1072   8.1  (0.6)   1072   8.6  (0.9)   885  22.3  (0.7)
  경기  11671  10.7  (0.3)  11671   8.5  (0.2)  9386  21.8  (0.4)
  강원   1841  14.4  (0.9)   1841  10.7  (0.6)  1374  21.4  (1.9)
  충북   1923  15.3  (1.3)   1923   8.7  (0.9)  1461  22.2  (1.3)
  충남   2272  14.9  (1.0)   2272  10.7  (0.7)  1684  22.4  (1.1)
  전북   2154  13.5  (1.0)   2154   8.5  (0.7)  1670  20.5  (1.3)
  전남   2146  13.4  (0.9)   2146  10.6  (0.7)  1624  22.2  (1.2)
  경북   2618  14.3  (0.9)   2618  10.6  (0.7)  1960  23.3  (1.1)
  경남   3392  12.8  (0.8)   3392   9.8  (0.5)  2608  20.2  (1.1)
  제주   1212  14.5  (1.4)   1212  12.0  (0.7)   884  21.0  (1.5)

### include_multiCol_df_list[2] : [신체] 흡연
    - T1 : 최근 30일 동안 1일 이상 일반담배(궐련)를 흡연한 사람의 분율
    - T2 : 최근 30일 동안 20일 이상 일반담배(궐련)를 흡연한 사람의 분율
    - T3 : 최근 30일 동안 1일 이상 일반담배(궐련)를 흡연하고, 하루 평균 일반담배(궐련) 흡연량이 10개비 이상인 사람의 분율
    - T4 : 현재 일반담배(궐련) 흡연자 중에서 최근 12개월 동안 담배를 끊으려고 시도한 사람의 분율
    - T5 : "구매 시도자 중에서 최근 30일 동안 편의점이나 가게 등에서 담배를 사려고 했을 때‘많이 노력’,‘조금만 노력’ 또는‘노력 없이도 쉽게’살 수 있었던 사람의 분율"
    - T6 : 최근 30일 동안 1일 이상 액상형 전자담배를 사용한 사람의 분율
    - T7 : 최근 30일 동안 1일 이상 궐련형 전자담배(가열담배, 예)아이코스, 글로, 릴 등)를 사용한 사람의 분율
    - T8 : 최근 7일 동안 학생의 집안에서 다른 사람이 피운 담배 연기를 마신 적이 있는 사람의 분율
    - T9 : "최근 7일 동안 집 또는 학교가 아닌 실내(상점, 식당, 쇼핑몰, 공연장, 피시(PC)방, 노래방 등)에서 다른 사람이 피운 담배 연기를 마신 적이 있는 사람의 분율 "

In [11]:
include_multiCol_df_list[2]

T1                 T2                  T3                T4        \
       표본 수   분율   표준오차   표본 수   분율    표준오차   표본 수   분율    표준오차 표본 수    분율   
시도                                                                           
  서울   7364  3.5  (0.3)   7364  2.1   (0.3)   7364  0.8   (0.1)  249  61.7   
  부산   3137  3.6  (0.6)   3137  2.1   (0.4)   3137  1.1   (0.2)  110  64.8   
  대구   2642  3.7  (0.5)   2642  2.4   (0.5)   2642  1.0   (0.2)   96  68.5   
  인천   2927  4.9  (0.7)   2927  2.7   (0.5)   2927  1.3   (0.3)  144  49.6   
  광주   2091  5.2  (0.9)   2091  3.4   (0.7)   2091  1.2  (0.5)*   96  70.5   
  대전   1853  4.3  (0.6)   1853  2.4   (0.5)   1853  1.1  (0.3)*   83  71.2   
  울산   1664  4.5  (0.7)   1664  2.0  (0.6)*   1664  1.0  (0.3)*   72  72.3   
  세종   1097  3.0  (0.4)   1097  1.6   (0.3)   1097  0.6  (0.2)*   38  77.5   
  경기  11934  4.0  (0.2)  11934  2.4   (0.2)  11934  0.8   (0.1)  463  62.3   
  강원   1895  5.0  (0.8)   1895  3.1   (0.6)   1895  0.9   (0.2)   87  77.9   
  충북   2001  5.1  (0.8)   2001  2.7   (0.6)   2001  1.1  (0.3)*  100  68.5   
  충남   2360  4.4  (0.6)   2360  2.6   (0.4)   2360  1.1   (0.2)  113  69.5   
  전북   2235  5.5  (0.6)   2235  2.8   (0.6)   2235  1.3   (0.3)  127  62.5   
  전남   2223  5.7  (0.8)   2223  3.5   (0.6)   2223  1.0  (0.3)*  116  73.4   
  경북   2704  5.0  (0.7)   2704  3.1   (0.5)   2704  1.4   (0.3)  141  66.3   
  경남   3495  3.6  (0.5)   3495  2.0   (0.4)   3495  0.9  (0.3)*  122  54.5   
  제주   1258  5.9  (0.8)   1258  4.0   (0.7)   1258  2.2   (0.4)   68  69.3   

              T5                  T6                 T7                 T8  \
       표준오차 표본 수    분율   표준오차   표본 수   분율   표준오차   표본 수   분율   표준오차   표본 수   
시도                                                                           
  서울  (2.7)  320  60.7  (2.6)   7364  2.9  (0.3)   7364  1.8  (0.2)   7364   
  부산  (4.6)  130  64.9  (5.8)   3137  2.5  (0.4)   3137  1.7  (0.3)   3137   
  대구  (3.2)  121  65.1  (3.6)   2642  2.5  (0.3)   2642  1.9  (0.4)   2642   
  인천  (4.7)  149  67.1  (4.5)   2927  3.3  (0.5)   2927  1.8  (0.3)   2927   
  광주  (2.9)   87  68.3  (4.3)   2091  4.0  (0.6)   2091  2.1  (0.4)   2091   
  대전  (4.7)   80  64.1  (6.4)   1853  2.9  (0.4)   1853  1.8  (0.3)   1853   
  울산  (5.4)   85  67.9  (6.3)   1664  3.7  (0.6)   1664  2.7  (0.5)   1664   
  세종  (5.8)   48  43.6  (7.3)   1097  1.8  (0.4)   1097  1.7  (0.3)   1097   
  경기  (2.4)  492  61.5  (2.3)  11934  3.0  (0.2)  11934  2.0  (0.2)  11934   
  강원  (4.6)   85  61.5  (5.5)   1895  3.9  (0.7)   1895  2.6  (0.5)   1895   
  충북  (4.1)   99  63.3  (4.7)   2001  3.6  (0.6)   2001  2.4  (0.4)   2001   
  충남  (6.3)  114  59.8  (4.8)   2360  3.1  (0.4)   2360  2.2  (0.4)   2360   
  전북  (4.2)  136  70.4  (3.9)   2235  4.0  (0.5)   2235  2.7  (0.4)   2235   
  전남  (5.4)  116  70.4  (4.3)   2223  4.2  (0.5)   2223  2.7  (0.4)   2223   
  경북  (3.1)  168  64.8  (4.3)   2704  3.6  (0.5)   2704  2.7  (0.4)   2704   
  경남  (4.2)  152  65.2  (5.7)   3495  2.6  (0.4)   3495  1.8  (0.3)   3495   
  제주  (5.4)   93  60.7  (3.0)   1258  4.5  (0.9)   1258  3.6  (0.7)   1258   

                      T9               
        분율   표준오차   표본 수    분율   표준오차  
시도                                     
  서울  17.1  (0.5)   7364  47.3  (0.7)  
  부산  19.7  (0.8)   3137  48.0  (1.0)  
  대구  20.1  (0.7)   2642  46.4  (1.3)  
  인천  22.0  (0.9)   2927  47.3  (1.2)  
  광주  21.9  (0.9)   2091  51.3  (1.3)  
  대전  20.5  (1.3)   1853  47.5  (1.5)  
  울산  21.1  (1.1)   1664  48.4  (1.2)  
  세종  13.6  (0.9)   1097  41.7  (1.5)  
  경기  20.6  (0.5)  11934  49.6  (0.6)  
  강원  23.9  (1.5)   1895  46.9  (1.9)  
  충북  24.9  (1.1)   2001  47.7  (1.7)  
  충남  22.2  (1.4)   2360  43.7  (1.9)  
  전북  23.1  (1.1)   2235  47.4  (1.0)  
  전남  23.4  (1.2)   2223  47.8  (1.7)  
  경북  21.8  (1.1)   2704  45.9  (1.5)  
  경남  21.6  (0.8)   3495  44.3  (1.3)  
  제주  23.6  (1.4)   1258  46.7  (1.7)

### include_multiCol_df_list[3] : [신체] 음주
    - T1 : 최근 30일 동안 1잔 이상 술을 마신 적이 있는 사람의 분율
    - T2 : 최근 30일 동안 1회 평균 음주량이 중등도 이상(남자: 소주 5잔 이상, 여자: 소주 3잔 이상)인 사람의 분율
    - T3 : 최근 30일 동안 1일 이상 정신을 잃거나 기억을 못 할 정도로 술을 마신 적이 있는 사람의 분율
    - T4 : "구매 시도자 중에서 최근 30일 동안 편의점이나 가게 등에서 술을 사려고 했을 때 ‘많이 노력’,‘조금만 노력’또는 ‘노력 없이도 쉽게’ 살 수 있었던 사람의 분율"

In [12]:
include_multiCol_df_list[3]

T1                  T2                 T3               T4        \
       표본 수    분율   표준오차   표본 수   분율   표준오차   표본 수   분율   표준오차 표본 수    분율   
시도                                                                          
  서울   7364   9.4  (0.5)   7364  4.1  (0.3)   7364  1.3  (0.2)  420  62.5   
  부산   3137   9.5  (0.6)   3137  4.3  (0.4)   3137  1.2  (0.2)  167  68.4   
  대구   2642   9.8  (0.7)   2642  4.6  (0.5)   2642  1.3  (0.2)  171  66.9   
  인천   2927  11.3  (0.9)   2927  5.3  (0.5)   2927  1.5  (0.2)  154  71.2   
  광주   2091  11.6  (1.0)   2091  5.7  (0.7)   2091  1.8  (0.4)  119  68.3   
  대전   1853  10.4  (0.8)   1853  5.5  (0.7)   1853  1.7  (0.3)  109  69.3   
  울산   1664  11.5  (1.5)   1664  5.2  (0.9)   1664  1.7  (0.3)   97  67.6   
  세종   1097   8.3  (0.9)   1097  3.0  (0.5)   1097  0.9  (0.2)   58  48.9   
  경기  11934  11.2  (0.4)  11934  4.7  (0.3)  11934  1.2  (0.1)  603  65.0   
  강원   1895  14.2  (1.0)   1895  6.1  (0.7)   1895  1.5  (0.2)   96  68.8   
  충북   2001  13.2  (1.2)   2001  5.8  (0.7)   2001  1.4  (0.3)  117  59.6   
  충남   2360  12.5  (1.1)   2360  5.9  (0.7)   2360  1.7  (0.2)  131  60.0   
  전북   2235  12.6  (0.9)   2235  5.8  (0.6)   2235  1.4  (0.3)  141  72.7   
  전남   2223  12.2  (1.1)   2223  5.6  (0.7)   2223  1.7  (0.4)  138  64.2   
  경북   2704  13.2  (0.9)   2704  6.3  (0.7)   2704  1.7  (0.3)  214  62.6   
  경남   3495  10.6  (0.8)   3495  4.3  (0.5)   3495  1.2  (0.2)  173  61.3   
  제주   1258  12.4  (1.5)   1258  7.0  (1.1)   1258  2.5  (0.6)   99  77.2   

             
       표준오차  
시도           
  서울  (2.4)  
  부산  (4.5)  
  대구  (3.0)  
  인천  (3.3)  
  광주  (4.3)  
  대전  (3.8)  
  울산  (5.9)  
  세종  (4.6)  
  경기  (2.2)  
  강원  (4.5)  
  충북  (4.6)  
  충남  (4.4)  
  전북  (3.2)  
  전남  (4.8)  
  경북  (3.7)  
  경남  (5.5)  
  제주  (3.5)

### include_multiCol_df_list[4] : [신체] 식생활
    - T1 : 최근 7일 동안 아침식사를 5일 이상 먹지 않은 사람의 분율
    - T2 : 최근 7일 동안 1일 1회 이상 과일(과일주스 제외)을 먹은 사람의 분율
    - T3 : 최근 7일 동안 3회 이상 피자, 햄버거, 치킨 같은 패스트푸드를 먹은 사람의 분율
    - T4 : 하루 평균 1일 5컵 이상 물을 마신 사람의 비율 

In [13]:
include_multiCol_df_list[4]

T1                  T2                  T3                  T4        \
       표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   
시도                                                                              
  서울   7364  42.3  (0.7)   7364  19.7  (0.6)   7364  29.7  (0.6)   7364  37.6   
  부산   3137  39.8  (1.1)   3137  18.6  (1.2)   3137  24.8  (0.8)   3137  38.8   
  대구   2642  40.0  (1.0)   2642  17.1  (0.8)   2642  27.0  (1.3)   2642  41.6   
  인천   2927  43.7  (1.4)   2927  13.8  (1.0)   2927  27.3  (0.9)   2927  37.6   
  광주   2091  43.2  (1.3)   2091  15.1  (0.8)   2091  27.4  (0.7)   2091  37.0   
  대전   1853  41.4  (1.5)   1853  15.9  (1.0)   1853  28.3  (1.4)   1853  37.0   
  울산   1664  38.1  (1.3)   1664  16.1  (0.9)   1664  26.8  (1.3)   1664  40.7   
  세종   1097  40.0  (1.2)   1097  22.4  (1.2)   1097  27.1  (1.3)   1097  35.8   
  경기  11934  40.2  (0.6)  11934  15.3  (0.4)  11934  26.8  (0.5)  11934  37.1   
  강원   1895  42.3  (1.0)   1895  14.4  (1.1)   1895  25.7  (1.0)   1895  38.7   
  충북   2001  40.4  (1.4)   2001  14.7  (0.8)   2001  26.0  (1.0)   2001  39.5   
  충남   2360  43.6  (1.6)   2360  15.0  (1.0)   2360  24.4  (0.9)   2360  37.1   
  전북   2235  43.6  (1.0)   2235  13.4  (0.7)   2235  28.9  (0.8)   2235  35.9   
  전남   2223  42.2  (1.6)   2223  14.0  (0.9)   2223  26.0  (1.0)   2223  34.2   
  경북   2704  40.1  (1.1)   2704  15.3  (0.8)   2704  25.2  (1.1)   2704  39.7   
  경남   3495  38.0  (1.2)   3495  15.0  (0.5)   3495  24.7  (0.8)   3495  37.2   
  제주   1258  41.9  (1.6)   1258  12.7  (0.7)   1258  26.8  (1.6)   1258  37.6   

             
       표준오차  
시도           
  서울  (0.8)  
  부산  (1.2)  
  대구  (1.5)  
  인천  (1.4)  
  광주  (1.4)  
  대전  (1.5)  
  울산  (1.7)  
  세종  (1.3)  
  경기  (0.5)  
  강원  (2.3)  
  충북  (2.2)  
  충남  (1.5)  
  전북  (1.2)  
  전남  (1.3)  
  경북  (1.4)  
  경남  (1.2)  
  제주  (2.8)

### include_multiCol_df_list[5] : [신체] 구강보건
    - T1 : 최근 7일 동안 학교에서 점심식사 후 칫솔질을 '항상' 또는 '대부분'한 사람의 분율
    - T2 : 최근 12개월 동안 실란트(치아홈메우기)를 받은 적이 있는 사람의 분율
    - T3 : "최근 12개월 동안 치아 통증 증상(차갑거나 뜨거운 음료 혹은 음식을 마시거나 먹을 때 치아가 아픔, 치아가 쑤시고 욱신거리고 아픔)을 한 가지 이상 경험한 적이 있는 사람의 분율"
    - T4 : 최근 12개월 동안 잇몸이 아프거나 피가 나는 경험을 한 사람의 분율
    - T5 : 최근 12개월 동안 치아가 깨지거나 부러진 사람 중에서 운동이나 사고로 인해 치아가 깨지거나 부러진 사람의 분율

In [14]:
include_multiCol_df_list[5]

T1                  T2                  T3                  T4        \
       표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   
시도                                                                              
  서울   7364  16.8  (0.9)   7364  29.9  (0.5)   7364  41.0  (0.6)   7364  20.4   
  부산   3137  25.9  (2.1)   3137  28.0  (1.1)   3137  39.7  (1.3)   3137  20.8   
  대구   2642  22.2  (1.9)   2642  24.2  (0.9)   2642  37.8  (0.8)   2642  16.4   
  인천   2927  23.7  (2.3)   2927  24.9  (0.8)   2927  41.4  (1.2)   2927  21.6   
  광주   2091  30.4  (2.7)   2091  29.9  (1.2)   2091  42.2  (1.2)   2091  20.0   
  대전   1853  30.1  (2.3)   1853  26.5  (1.1)   1853  39.5  (1.4)   1853  21.5   
  울산   1664  31.7  (2.4)   1664  28.5  (1.4)   1664  39.0  (1.4)   1664  18.4   
  세종   1097  44.5  (2.4)   1097  28.6  (1.6)   1097  39.3  (1.3)   1097  19.5   
  경기  11934  24.3  (0.9)  11934  27.5  (0.5)  11934  41.0  (0.5)  11934  20.8   
  강원   1895  32.2  (3.1)   1895  24.0  (1.4)   1895  38.3  (0.9)   1895  17.5   
  충북   2001  37.7  (2.4)   2001  23.9  (1.1)   2001  41.1  (1.6)   2001  20.3   
  충남   2360  32.3  (2.8)   2360  26.5  (1.2)   2360  40.1  (1.5)   2360  18.3   
  전북   2235  31.2  (2.6)   2235  30.3  (1.3)   2235  42.2  (0.9)   2235  21.7   
  전남   2223  34.6  (2.9)   2223  24.0  (1.0)   2223  41.4  (1.4)   2223  18.8   
  경북   2704  33.1  (3.2)   2704  27.7  (0.9)   2704  40.8  (1.5)   2704  18.3   
  경남   3495  32.7  (2.9)   3495  25.4  (1.1)   3495  39.1  (1.1)   3495  19.8   
  제주   1258  28.4  (2.8)   1258  26.9  (1.8)   1258  45.0  (2.8)   1258  21.5   

               T5               
       표준오차  표본 수    분율   표준오차  
시도                              
  서울  (0.6)   733  35.1  (1.7)  
  부산  (1.0)   309  32.0  (3.5)  
  대구  (0.7)   266  34.4  (2.9)  
  인천  (0.7)   329  38.7  (3.4)  
  광주  (1.0)   246  35.4  (3.2)  
  대전  (1.0)   183  30.6  (3.4)  
  울산  (0.9)   166  29.9  (3.9)  
  세종  (0.8)   110  42.7  (4.7)  
  경기  (0.4)  1203  34.6  (1.5)  
  강원  (0.9)   206  32.3  (3.4)  
  충북  (1.4)   251  31.9  (3.1)  
  충남  (0.8)   273  30.9  (2.7)  
  전북  (0.9)   252  36.0  (3.1)  
  전남  (1.2)   283  27.9  (2.1)  
  경북  (1.3)   340  34.1  (2.2)  
  경남  (0.8)   379  38.4  (2.6)  
  제주  (1.8)   148  37.4  (4.9)

### include_multiCol_df_list[6] : [신체] 개인위생
    - T1 : 최근 7일 동안 학교에서 식사하기 전 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T2 : 최근 7일 동안 학교에서 화장실 다녀온 후 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T3 : 최근 7일 동안 집에서 식사하기 전 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T4 : 최근 7일 동안 집에서 화장실 다녀온 후 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T5 : 최근 7일 동안 외출 후 집에 돌아왔을 때 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율

In [15]:
include_multiCol_df_list[6]

T1                  T2                  T3                  T4        \
       표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   
시도                                                                              
  서울   7364  44.4  (0.8)   7364  88.1  (0.5)   7364  63.4  (0.6)   7364  87.1   
  부산   3137  48.2  (1.4)   3137  89.7  (0.6)   3137  64.9  (1.0)   3137  87.9   
  대구   2642  47.8  (1.6)   2642  88.5  (0.7)   2642  65.7  (0.9)   2642  87.9   
  인천   2927  45.6  (1.1)   2927  86.8  (0.8)   2927  62.5  (0.9)   2927  86.1   
  광주   2091  37.8  (1.6)   2091  81.3  (1.5)   2091  58.8  (1.3)   2091  83.0   
  대전   1853  44.5  (1.3)   1853  86.1  (1.2)   1853  61.0  (1.3)   1853  85.3   
  울산   1664  47.9  (1.6)   1664  88.8  (1.0)   1664  65.7  (1.0)   1664  87.9   
  세종   1097  46.5  (1.6)   1097  85.1  (0.9)   1097  62.2  (1.4)   1097  85.3   
  경기  11934  47.0  (0.8)  11934  88.0  (0.3)  11934  63.0  (0.5)  11934  86.9   
  강원   1895  41.7  (1.7)   1895  87.5  (1.0)   1895  58.5  (1.3)   1895  85.8   
  충북   2001  42.3  (1.4)   2001  85.3  (1.3)   2001  60.6  (1.4)   2001  84.4   
  충남   2360  43.7  (1.4)   2360  86.1  (0.9)   2360  60.4  (1.3)   2360  85.3   
  전북   2235  40.5  (1.0)   2235  84.3  (1.2)   2235  61.3  (0.8)   2235  84.3   
  전남   2223  44.5  (1.7)   2223  80.8  (1.2)   2223  60.8  (1.1)   2223  79.7   
  경북   2704  46.6  (1.5)   2704  86.4  (0.8)   2704  60.4  (1.2)   2704  84.1   
  경남   3495  51.0  (1.4)   3495  85.1  (0.9)   3495  64.1  (0.9)   3495  83.9   
  제주   1258  39.6  (1.9)   1258  87.5  (1.3)   1258  56.7  (1.6)   1258  81.2   

                T5               
       표준오차   표본 수    분율   표준오차  
시도                               
  서울  (0.4)   7364  84.7  (0.5)  
  부산  (0.5)   3137  85.0  (0.7)  
  대구  (0.5)   2642  84.2  (0.6)  
  인천  (0.5)   2927  83.6  (0.5)  
  광주  (1.0)   2091  75.6  (1.1)  
  대전  (1.1)   1853  81.1  (1.1)  
  울산  (0.7)   1664  83.4  (0.8)  
  세종  (0.8)   1097  80.8  (0.9)  
  경기  (0.3)  11934  84.0  (0.3)  
  강원  (1.2)   1895  80.0  (1.2)  
  충북  (1.0)   2001  79.3  (0.9)  
  충남  (0.8)   2360  80.2  (0.8)  
  전북  (0.9)   2235  78.3  (1.0)  
  전남  (1.1)   2223  74.9  (1.2)  
  경북  (0.6)   2704  78.8  (0.8)  
  경남  (0.9)   3495  80.3  (0.7)  
  제주  (1.2)   1258  71.2  (0.7)

### include_multiCol_df_list[7] : [신체] 아토피/천식
    - T1 : 태어나서 지금까지 천식이라고 의사에게 진단받은 적이 있는 사람의 분율
    - T2 : 태어나서 지금까지 알레르기비염이라고 의사에게 진단받은 적이 있는 사람의 분율
    - T3 : 태어나서 지금까지 아토피피부염(습진 또는 태열)이라고 의사에게 진단받은 적이 있는 사람의 분율

In [16]:
include_multiCol_df_list[7]

T1                 T2                  T3             
       표본 수   분율   표준오차   표본 수    분율   표준오차   표본 수    분율   표준오차
시도                                                             
  서울   7364  5.9  (0.3)   7364  38.6  (0.7)   7364  22.8  (0.6)
  부산   3137  4.6  (0.3)   3137  34.2  (1.2)   3137  18.2  (0.6)
  대구   2642  6.1  (0.5)   2642  38.0  (1.1)   2642  21.4  (0.7)
  인천   2927  4.5  (0.4)   2927  35.3  (1.2)   2927  22.6  (1.0)
  광주   2091  5.2  (0.5)   2091  37.2  (1.3)   2091  23.1  (0.8)
  대전   1853  5.2  (0.6)   1853  39.5  (1.6)   1853  23.3  (1.1)
  울산   1664  4.7  (0.5)   1664  37.7  (1.5)   1664  19.5  (1.0)
  세종   1097  4.1  (0.6)   1097  39.1  (1.3)   1097  22.9  (1.5)
  경기  11934  5.8  (0.2)  11934  41.5  (0.5)  11934  22.4  (0.4)
  강원   1895  4.9  (0.5)   1895  33.8  (1.7)   1895  19.8  (0.9)
  충북   2001  5.3  (0.7)   2001  33.5  (1.0)   2001  20.9  (0.9)
  충남   2360  5.8  (0.4)   2360  38.5  (1.3)   2360  18.6  (1.0)
  전북   2235  5.7  (0.5)   2235  36.6  (1.1)   2235  22.9  (1.0)
  전남   2223  5.1  (0.5)   2223  30.5  (1.3)   2223  21.6  (0.9)
  경북   2704  5.8  (0.5)   2704  34.8  (1.5)   2704  22.1  (1.2)
  경남   3495  4.8  (0.3)   3495  35.9  (1.2)   3495  21.1  (0.7)
  제주   1258  4.9  (0.6)   1258  34.9  (0.8)   1258  21.5  (1.0)

### include_multiCol_df_list[8] : [정신건강] 정신건강
    - T1 : [스트레스 인지율] 평상시 스트레스를 ‘대단히 많이’ 또는 ‘많이’ 느끼는 사람의 분율
    - T2 : [우울감 경험률] 최근 12개월 동안 2주 내내 일상생활을 중단할 정도로 슬프거나 절망감을 느낀 적이 있는 사람의 분율
    - T3 : [자살 생각률] 최근 12개월 동안 심각하게 자살을 생각한 적이 있는 사람의 분율
    - T4 : [자살 계획률] 최근 12개월 동안 자살하기 위해 구체적인 계획을 세운 적이 있는 사람의 분율
    - T5 : [주중 평균 수면시간] 최근 7일 동안 주중(월~금) 수면시간의 평균
    - T6 : [주관적 행복감 경험률] 평상시‘매우’또는‘약간’행복한 편이라고 생각한 사람의 분율 [가점]
    - T7 : [외로움 경험률] 최근 12개월 동안‘자주’또는 '항상’외로움을 느낀 사람의 분율
    - T8 : [중증도 이상 범불안장애 경험률] 범불안장애 선별도구(GAD-7) 총점 21점 중 10점(중등도 불안)이상인 사람의 분율

In [17]:
include_multiCol_df_list[8]

T1                  T2                  T3                  T4       \
       표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수   분율   
시도                                                                             
  서울   7364  39.6  (0.9)   7364  27.5  (0.7)   7364  14.5  (0.5)   7364  5.6   
  부산   3137  32.6  (1.1)   3137  21.5  (0.8)   3137  10.7  (0.7)   3137  4.4   
  대구   2642  35.0  (1.1)   2642  24.9  (0.9)   2642  11.8  (0.9)   2642  5.1   
  인천   2927  36.9  (1.3)   2927  26.3  (1.1)   2927  14.0  (1.0)   2927  4.8   
  광주   2091  37.4  (1.3)   2091  25.2  (1.0)   2091  14.1  (0.8)   2091  5.2   
  대전   1853  38.5  (1.5)   1853  26.6  (1.3)   1853  13.6  (1.0)   1853  6.0   
  울산   1664  34.3  (1.5)   1664  25.9  (1.5)   1664   9.9  (0.9)   1664  4.4   
  세종   1097  36.1  (1.2)   1097  27.0  (1.3)   1097  13.1  (0.7)   1097  4.5   
  경기  11934  39.0  (0.7)  11934  27.1  (0.5)  11934  14.9  (0.4)  11934  5.8   
  강원   1895  36.8  (1.7)   1895  25.0  (1.3)   1895  12.6  (0.9)   1895  4.1   
  충북   2001  36.8  (2.0)   2001  25.5  (1.7)   2001  12.6  (1.0)   2001  4.2   
  충남   2360  36.7  (1.0)   2360  25.7  (1.1)   2360  11.7  (0.8)   2360  4.8   
  전북   2235  40.0  (1.2)   2235  28.6  (1.3)   2235  15.4  (0.9)   2235  6.3   
  전남   2223  35.2  (1.8)   2223  24.2  (1.4)   2223  12.6  (0.9)   2223  5.3   
  경북   2704  34.8  (1.3)   2704  25.7  (1.4)   2704  11.3  (0.9)   2704  4.7   
  경남   3495  33.5  (1.2)   3495  22.7  (0.9)   3495  11.6  (0.8)   3495  4.5   
  제주   1258  37.3  (2.5)   1258  26.4  (1.7)   1258  13.9  (1.8)   1258  6.9   

                T5                 T6                  T7                  T8  \
       표준오차   표본 수   분율   표준오차   표본 수    분율   표준오차   표본 수    분율   표준오차   표본 수   
시도                                                                              
  서울  (0.3)   6856  5.9  (0.0)   7364  56.7  (0.7)   7364  19.0  (0.6)   7364   
  부산  (0.5)   2887  6.1  (0.0)   3137  61.4  (1.0)   3137  14.1  (0.9)   3137   
  대구  (0.4)   2447  6.1  (0.1)   2642  59.4  (1.0)   2642  16.7  (0.9)   2642   
  인천  (0.5)   2701  6.3  (0.1)   2927  57.6  (1.2)   2927  17.9  (0.9)   2927   
  광주  (0.6)   1964  6.2  (0.0)   2091  56.1  (1.3)   2091  17.8  (0.8)   2091   
  대전  (0.7)   1709  6.0  (0.1)   1853  54.5  (1.2)   1853  18.3  (1.4)   1853   
  울산  (0.6)   1550  6.4  (0.1)   1664  62.6  (1.2)   1664  14.1  (1.0)   1664   
  세종  (0.7)   1017  6.3  (0.0)   1097  58.8  (1.4)   1097  19.0  (0.9)   1097   
  경기  (0.2)  11026  6.3  (0.0)  11934  56.9  (0.6)  11934  20.1  (0.4)  11934   
  강원  (0.5)   1776  6.5  (0.1)   1895  56.3  (1.2)   1895  17.8  (1.4)   1895   
  충북  (0.5)   1859  6.4  (0.0)   2001  58.8  (1.6)   2001  18.1  (1.5)   2001   
  충남  (0.4)   2178  6.4  (0.1)   2360  58.4  (1.3)   2360  16.8  (0.9)   2360   
  전북  (0.6)   2070  6.4  (0.1)   2235  54.5  (1.0)   2235  20.0  (1.1)   2235   
  전남  (0.6)   2054  6.3  (0.1)   2223  60.2  (1.4)   2223  16.7  (1.0)   2223   
  경북  (0.4)   2513  6.2  (0.1)   2704  58.6  (1.2)   2704  15.7  (1.0)   2704   
  경남  (0.4)   3232  6.3  (0.1)   3495  59.5  (1.2)   3495  15.9  (0.9)   3495   
  제주  (1.0)   1154  6.4  (0.1)   1258  60.0  (1.9)   1258  18.7  (1.4)   1258   

                   
        분율   표준오차  
시도                 
  서울  13.4  (0.5)  
  부산  10.9  (0.8)  
  대구  10.1  (0.7)  
  인천  12.5  (0.7)  
  광주  13.4  (0.7)  
  대전  12.1  (1.0)  
  울산  11.2  (1.0)  
  세종  12.7  (0.9)  
  경기  13.5  (0.4)  
  강원  11.0  (0.9)  
  충북  11.8  (0.9)  
  충남  12.0  (1.0)  
  전북  14.8  (1.0)  
  전남  11.9  (1.0)  
  경북  12.3  (0.9)  
  경남  11.4  (0.6)  
  제주  13.4  (1.6)

# 순위부여 함수적용 (감・가점 적용 전)

    [분율에 대한 지역별 및 항목별 순위]
    - T1 : 신체활동
    - T2 : 비만
    - T3 : 흡연
    - T4 : 음주
    - T5 : 식생활
    - T6 : 구강보건
    - T7 : 개인위생
    - T8 : 아토피/천식
    - T9 : 정신건강
 

In [18]:
final_result_df = give_rank_to_df(include_multiCol_df_list)
final_result_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9
시도,,,,,,,,,
서울,7,9,16,14,1,6,5,1,4
부산,6,6,14,12,12,11,1,17,17
대구,14,14,13,9,2,17,2,4,15
인천,11,3,9,9,3,5,6,10,6
광주,9,17,2,5,4,3,17,5,8
대전,13,16,11,7,5,12,9,2,7
울산,1,6,7,11,6,15,3,14,9
세종,16,1,17,16,8,3,8,7,5
경기,8,15,12,15,13,7,4,3,2


# 순위부여 함수적용 (감・가점 적용 후)
    [분율에 대한 지역별 및 항목별 순위]
    - T1 : 신체활동
    - T2 : 비만
    - T3 : 흡연
    - T4 : 음주
    - T5 : 식생활
    - T6 : 구강보건
    - T7 : 개인위생
    - T8 : 아토피/천식
    - T9 : 정신건강

In [19]:
plus_col_list = [ [('T1', '분율'), ('T2', '분율') , ('T3', '분율')],
                 [], [], [],
                 [('T2', '분율'), ('T4', '분율')],
                 [('T1', '분율'), ('T2', '분율')],
                 [('T1', '분율'), ('T2', '분율'), ('T3', '분율'), ('T4', '분율'), ('T5', '분율')],
                 [], 
                 [6] ]

In [20]:
res = give_rank_to_df_2( include_multiCol_df_list, plus_col_list )
res

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16
시도,,,,,,,,,,,,,,,,
서울,11,9,16,14,7,5,13,1,2,2,4,5,17,5,3,3
부산,12,6,14,12,17,9,17,17,17,17,17,14,14,16,16,16
대구,4,14,13,9,14,11,16,4,13,14,13,8,14,12,12,17
인천,7,3,9,9,2,1,12,10,7,7,6,9,7,7,8,7
광주,9,17,2,5,3,6,1,4,5,12,5,7,12,3,9,3
대전,5,16,10,7,6,7,9,1,4,5,8,3,16,1,6,9
울산,17,6,7,9,16,17,15,13,15,8,1,14,2,17,16,14
세종,2,1,17,16,11,13,10,7,11,4,9,12,7,10,3,6
경기,10,14,12,15,9,3,14,1,3,3,3,4,7,6,1,2


# 결과 더블 체크

In [21]:
plus_col_list = [ [('T1', '분율'), ('T2', '분율') , ('T3', '분율')],
                 [], [], [],
                 [('T2', '분율'), ('T4', '분율')],
                 [('T1', '분율'), ('T2', '분율')],
                 [('T1', '분율'), ('T2', '분율'), ('T3', '분율'), ('T4', '분율'), ('T5', '분율')],
                 [], 
                 [6] ]

In [22]:
# 가점요소 없는 데이터
defalut_df = include_multiCol_df_list[2].iloc[:, 1::3]
plus_df = defalut_df[plus_col_list[2]].rank(method='min')
plus_df

시도
서울
부산
대구
인천
광주
대전
울산
세종
경기
강원


In [23]:
defalut_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9
,분율,분율,분율,분율,분율,분율,분율,분율,분율
시도,,,,,,,,,
서울,3.5,2.1,0.8,61.7,60.7,2.9,1.8,17.1,47.3
부산,3.6,2.1,1.1,64.8,64.9,2.5,1.7,19.7,48.0
대구,3.7,2.4,1.0,68.5,65.1,2.5,1.9,20.1,46.4
인천,4.9,2.7,1.3,49.6,67.1,3.3,1.8,22.0,47.3
광주,5.2,3.4,1.2,70.5,68.3,4.0,2.1,21.9,51.3
대전,4.3,2.4,1.1,71.2,64.1,2.9,1.8,20.5,47.5
울산,4.5,2.0,1.0,72.3,67.9,3.7,2.7,21.1,48.4
세종,3.0,1.6,0.6,77.5,43.6,1.8,1.7,13.6,41.7


In [24]:
minus_df = defalut_df.loc[:, ~defalut_df.columns.isin(plus_col_list[2])].rank(method='min',
                                                                           ascending=False)
minus_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9
,분율,분율,분율,분율,분율,분율,분율,분율,분율
시도,,,,,,,,,
서울,16.0,13.0,15.0,15.0,14.0,12.0,12.0,16.0,9.0
부산,14.0,13.0,6.0,12.0,8.0,15.0,16.0,15.0,4.0
대구,13.0,10.0,10.0,9.0,7.0,15.0,11.0,14.0,13.0
인천,8.0,7.0,3.0,17.0,5.0,9.0,12.0,7.0,9.0
광주,4.0,3.0,5.0,6.0,3.0,3.0,9.0,8.0,1.0
대전,11.0,10.0,6.0,5.0,10.0,12.0,12.0,13.0,7.0
울산,9.0,15.0,10.0,4.0,4.0,6.0,2.0,11.0,3.0
세종,17.0,17.0,17.0,2.0,17.0,17.0,16.0,17.0,17.0


In [25]:
first_result = pd.concat([plus_df, minus_df], axis=1)
first_result

,T1,T2,T3,T4,T5,T6,T7,T8,T9
,분율,분율,분율,분율,분율,분율,분율,분율,분율
시도,,,,,,,,,
서울,16.0,13.0,15.0,15.0,14.0,12.0,12.0,16.0,9.0
부산,14.0,13.0,6.0,12.0,8.0,15.0,16.0,15.0,4.0
대구,13.0,10.0,10.0,9.0,7.0,15.0,11.0,14.0,13.0
인천,8.0,7.0,3.0,17.0,5.0,9.0,12.0,7.0,9.0
광주,4.0,3.0,5.0,6.0,3.0,3.0,9.0,8.0,1.0
대전,11.0,10.0,6.0,5.0,10.0,12.0,12.0,13.0,7.0
울산,9.0,15.0,10.0,4.0,4.0,6.0,2.0,11.0,3.0
세종,17.0,17.0,17.0,2.0,17.0,17.0,16.0,17.0,17.0


In [26]:
second_result = first_result.sum(axis=1).rank(method='min').astype(int)
second_result

시도
  서울    16
  부산    14
  대구    13
  인천     9
  광주     2
  대전    10
  울산     7
  세종    17
  경기    12
  강원     6
  충북     5
  충남    11
  전북     4
  전남     1
  경북     7
  경남    15
  제주     2
dtype: int64

# 개인건강 최종산출물

In [27]:
df = pd.concat(include_multiCol_df_list, axis=1).iloc[:, 1::3]
df

,T1,T2,T3,T1,T2,T3,T1,T2,T3,T4,T5,T6,T7,T8,T9,T1,T2,T3,T4,T1,T2,T3,T4,T1,T2,T3,T4,T5,T1,T2,T3,T4,T5,T1,T2,T3,T1,T2,T3,T4,T5,T6,T7,T8
,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율,분율
시도,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
서울,17.7,42.0,24.6,11.0,8.2,22.5,3.5,2.1,0.8,61.7,60.7,2.9,1.8,17.1,47.3,9.4,4.1,1.3,62.5,42.3,19.7,29.7,37.6,16.8,29.9,41.0,20.4,35.1,44.4,88.1,63.4,87.1,84.7,5.9,38.6,22.8,39.6,27.5,14.5,5.6,5.9,56.7,19.0,13.4
부산,17.1,41.6,26.0,12.1,9.2,21.8,3.6,2.1,1.1,64.8,64.9,2.5,1.7,19.7,48.0,9.5,4.3,1.2,68.4,39.8,18.6,24.8,38.8,25.9,28.0,39.7,20.8,32.0,48.2,89.7,64.9,87.9,85.0,4.6,34.2,18.2,32.6,21.5,10.7,4.4,6.1,61.4,14.1,10.9
대구,16.1,40.1,25.4,12.6,8.6,19.2,3.7,2.4,1.0,68.5,65.1,2.5,1.9,20.1,46.4,9.8,4.6,1.3,66.9,40.0,17.1,27.0,41.6,22.2,24.2,37.8,16.4,34.4,47.8,88.5,65.7,87.9,84.2,6.1,38.0,21.4,35.0,24.9,11.8,5.1,6.1,59.4,16.7,10.1
인천,18.7,39.0,24.7,12.6,9.4,22.0,4.9,2.7,1.3,49.6,67.1,3.3,1.8,22.0,47.3,11.3,5.3,1.5,71.2,43.7,13.8,27.3,37.6,23.7,24.9,41.4,21.6,38.7,45.6,86.8,62.5,86.1,83.6,4.5,35.3,22.6,36.9,26.3,14.0,4.8,6.3,57.6,17.9,12.5
광주,15.8,41.1,26.0,10.6,8.4,21.4,5.2,3.4,1.2,70.5,68.3,4.0,2.1,21.9,51.3,11.6,5.7,1.8,68.3,43.2,15.1,27.4,37.0,30.4,29.9,42.2,20.0,35.4,37.8,81.3,58.8,83.0,75.6,5.2,37.2,23.1,37.4,25.2,14.1,5.2,6.2,56.1,17.8,13.4
대전,17.6,40.0,24.6,11.0,8.5,20.8,4.3,2.4,1.1,71.2,64.1,2.9,1.8,20.5,47.5,10.4,5.5,1.7,69.3,41.4,15.9,28.3,37.0,30.1,26.5,39.5,21.5,30.6,44.5,86.1,61.0,85.3,81.1,5.2,39.5,23.3,38.5,26.6,13.6,6.0,6.0,54.5,18.3,12.1
울산,19.1,45.7,26.9,11.3,8.7,22.1,4.5,2.0,1.0,72.3,67.9,3.7,2.7,21.1,48.4,11.5,5.2,1.7,67.6,38.1,16.1,26.8,40.7,31.7,28.5,39.0,18.4,29.9,47.9,88.8,65.7,87.9,83.4,4.7,37.7,19.5,34.3,25.9,9.9,4.4,6.4,62.6,14.1,11.2
세종,15.3,41.7,24.4,8.1,8.6,22.3,3.0,1.6,0.6,77.5,43.6,1.8,1.7,13.6,41.7,8.3,3.0,0.9,48.9,40.0,22.4,27.1,35.8,44.5,28.6,39.3,19.5,42.7,46.5,85.1,62.2,85.3,80.8,4.1,39.1,22.9,36.1,27.0,13.1,4.5,6.3,58.8,19.0,12.7


## 컬럼설명
    - T1 : "최근 7일 동안 운동 종류 상관없이 심장박동이 평상시보다 증가하거나, 숨이 찬 정도의 신체활동을 하루에 총합이 60분 이상 한 날이 5일 이상인 사람의 분율"
    - T2 : "최근 7일 동안 고강도 신체활동(조깅(달리기), 축구, 농구, 단식(1:1) 테니스, 태권도, 등산, 빠른 속도의 자전거 타기, 빠른 수영, 줄넘기, 무거운 물건 나르기, 에어로빅 등)을 3일 이상한 사람의 분율"
    - T3 : 최근 7일 동안 근력강화운동(팔굽혀펴기, 윗몸일으키기, 역기 들기, 아령, 철봉, 평행봉 등)을 3일 이상 한 사람의 분율
    - T4 : 2017년 소아청소년 성장도표 연령별 체질량지수 기준 95백분위수 이상인 사람의 분율
    - T5 : 2017년 소아청소년 성장도표 연령별 체질량지수 기준 85백분위수 이상에서 95백분위수 미만인 사람의 분율
    - T6 : 2017년 소아청소년 성장도표 연령별 체질량지수 기준 85백분위수 미만인 자 중에서 자신의 체형을 살이 찐 편이라고 인지하는 사람의 분율
    - T7 : 최근 30일 동안 1일 이상 일반담배(궐련)를 흡연한 사람의 분율
    - T8 : 최근 30일 동안 20일 이상 일반담배(궐련)를 흡연한 사람의 분율
    - T9 : 최근 30일 동안 1일 이상 일반담배(궐련)를 흡연하고, 하루 평균 일반담배(궐련) 흡연량이 10개비 이상인 사람의 분율
    - T10 : 현재 일반담배(궐련) 흡연자 중에서 최근 12개월 동안 담배를 끊으려고 시도한 사람의 분율
    - T11 : "구매 시도자 중에서 최근 30일 동안 편의점이나 가게 등에서 담배를 사려고 했을 때‘많이 노력’,‘조금만 노력’ 또는‘노력 없이도 쉽게’살 수 있었던 사람의 분율"
    - T12 : 최근 30일 동안 1일 이상 액상형 전자담배를 사용한 사람의 분율
    - T13 : 최근 30일 동안 1일 이상 궐련형 전자담배(가열담배, 예)아이코스, 글로, 릴 등)를 사용한 사람의 분율
    - T14 : 최근 7일 동안 학생의 집안에서 다른 사람이 피운 담배 연기를 마신 적이 있는 사람의 분율
    - T15 : "최근 7일 동안 집 또는 학교가 아닌 실내(상점, 식당, 쇼핑몰, 공연장, 피시(PC)방, 노래방 등)에서 다른 사람이 피운 담배 연기를 마신 적이 있는 사람의 분율 "
    - T16 : 최근 30일 동안 1잔 이상 술을 마신 적이 있는 사람의 분율
    - T17 : 최근 30일 동안 1회 평균 음주량이 중등도 이상(남자: 소주 5잔 이상, 여자: 소주 3잔 이상)인 사람의 분율
    - T18 : 최근 30일 동안 1일 이상 정신을 잃거나 기억을 못 할 정도로 술을 마신 적이 있는 사람의 분율
    - T19 : "구매 시도자 중에서 최근 30일 동안 편의점이나 가게 등에서 술을 사려고 했을 때 ‘많이 노력’,‘조금만 노력’또는 ‘노력 없이도 쉽게’ 살 수 있었던 사람의 분율"
    - T20 : 최근 7일 동안 아침식사를 5일 이상 먹지 않은 사람의 분율
    - T21 : 최근 7일 동안 1일 1회 이상 과일(과일주스 제외)을 먹은 사람의 분율
    - T22 : 최근 7일 동안 3회 이상 피자, 햄버거, 치킨 같은 패스트푸드를 먹은 사람의 분율
    - T23 : 하루 평균 1일 5컵 이상 물을 마신 사람의 비율 
    - T24 : 최근 7일 동안 학교에서 점심식사 후 칫솔질을 '항상' 또는 '대부분'한 사람의 분율
    - T25 : 최근 12개월 동안 실란트(치아홈메우기)를 받은 적이 있는 사람의 분율
    - T26 : "최근 12개월 동안 치아 통증 증상(차갑거나 뜨거운 음료 혹은 음식을 마시거나 먹을 때 치아가 아픔, 치아가 쑤시고 욱신거리고 아픔)을 한 가지 이상 경험한 적이 있는 사람의 분율"
    - T27 : 최근 12개월 동안 잇몸이 아프거나 피가 나는 경험을 한 사람의 분율
    - T28 : 최근 12개월 동안 치아가 깨지거나 부러진 사람 중에서 운동이나 사고로 인해 치아가 깨지거나 부러진 사람의 분율
    - T29 : 최근 7일 동안 학교에서 식사하기 전 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T30 : 최근 7일 동안 학교에서 화장실 다녀온 후 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T31 : 최근 7일 동안 집에서 식사하기 전 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T32 : 최근 7일 동안 집에서 화장실 다녀온 후 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T33 : 최근 7일 동안 외출 후 집에 돌아왔을 때 비누를 이용한 손씻기를 ‘항상’ 또는 ‘대부분’ 한 사람의 분율
    - T34 : 태어나서 지금까지 천식이라고 의사에게 진단받은 적이 있는 사람의 분율
    - T35 : 태어나서 지금까지 알레르기비염이라고 의사에게 진단받은 적이 있는 사람의 분율
    - T36 : 태어나서 지금까지 아토피피부염(습진 또는 태열)이라고 의사에게 진단받은 적이 있는 사람의 분율
    - T37 : [스트레스 인지율] 평상시 스트레스를 ‘대단히 많이’ 또는 ‘많이’ 느끼는 사람의 분율
    - T38 : [우울감 경험률] 최근 12개월 동안 2주 내내 일상생활을 중단할 정도로 슬프거나 절망감을 느낀 적이 있는 사람의 분율
    - T39 : [자살 생각률] 최근 12개월 동안 심각하게 자살을 생각한 적이 있는 사람의 분율
    - T40 : [자살 계획률] 최근 12개월 동안 자살하기 위해 구체적인 계획을 세운 적이 있는 사람의 분율
    - T41 : [주중 평균 수면시간] 최근 7일 동안 주중(월~금) 수면시간의 평균
    - T42 : [주관적 행복감 경험률] 평상시‘매우’또는‘약간’행복한 편이라고 생각한 사람의 분율 [가점]
    - T43 : [외로움 경험률] 최근 12개월 동안‘자주’또는 '항상’외로움을 느낀 사람의 분율
    - T44 : [중증도 이상 범불안장애 경험률] 범불안장애 선별도구(GAD-7) 총점 21점 중 10점(중등도 불안)이상인 사람의 분율

### (1) 전체 세부항목들에 대한 분율


In [28]:
df_1 = df.copy()
num_col = len(df_1.columns)
multi_col = [f'T{j + 1}' for j in range(num_col)]
df_1.columns = multi_col
df_1

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29,T30,T31,T32,T33,T34,T35,T36,T37,T38,T39,T40,T41,T42,T43,T44
시도,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
서울,17.7,42.0,24.6,11.0,8.2,22.5,3.5,2.1,0.8,61.7,60.7,2.9,1.8,17.1,47.3,9.4,4.1,1.3,62.5,42.3,19.7,29.7,37.6,16.8,29.9,41.0,20.4,35.1,44.4,88.1,63.4,87.1,84.7,5.9,38.6,22.8,39.6,27.5,14.5,5.6,5.9,56.7,19.0,13.4
부산,17.1,41.6,26.0,12.1,9.2,21.8,3.6,2.1,1.1,64.8,64.9,2.5,1.7,19.7,48.0,9.5,4.3,1.2,68.4,39.8,18.6,24.8,38.8,25.9,28.0,39.7,20.8,32.0,48.2,89.7,64.9,87.9,85.0,4.6,34.2,18.2,32.6,21.5,10.7,4.4,6.1,61.4,14.1,10.9
대구,16.1,40.1,25.4,12.6,8.6,19.2,3.7,2.4,1.0,68.5,65.1,2.5,1.9,20.1,46.4,9.8,4.6,1.3,66.9,40.0,17.1,27.0,41.6,22.2,24.2,37.8,16.4,34.4,47.8,88.5,65.7,87.9,84.2,6.1,38.0,21.4,35.0,24.9,11.8,5.1,6.1,59.4,16.7,10.1
인천,18.7,39.0,24.7,12.6,9.4,22.0,4.9,2.7,1.3,49.6,67.1,3.3,1.8,22.0,47.3,11.3,5.3,1.5,71.2,43.7,13.8,27.3,37.6,23.7,24.9,41.4,21.6,38.7,45.6,86.8,62.5,86.1,83.6,4.5,35.3,22.6,36.9,26.3,14.0,4.8,6.3,57.6,17.9,12.5
광주,15.8,41.1,26.0,10.6,8.4,21.4,5.2,3.4,1.2,70.5,68.3,4.0,2.1,21.9,51.3,11.6,5.7,1.8,68.3,43.2,15.1,27.4,37.0,30.4,29.9,42.2,20.0,35.4,37.8,81.3,58.8,83.0,75.6,5.2,37.2,23.1,37.4,25.2,14.1,5.2,6.2,56.1,17.8,13.4
대전,17.6,40.0,24.6,11.0,8.5,20.8,4.3,2.4,1.1,71.2,64.1,2.9,1.8,20.5,47.5,10.4,5.5,1.7,69.3,41.4,15.9,28.3,37.0,30.1,26.5,39.5,21.5,30.6,44.5,86.1,61.0,85.3,81.1,5.2,39.5,23.3,38.5,26.6,13.6,6.0,6.0,54.5,18.3,12.1
울산,19.1,45.7,26.9,11.3,8.7,22.1,4.5,2.0,1.0,72.3,67.9,3.7,2.7,21.1,48.4,11.5,5.2,1.7,67.6,38.1,16.1,26.8,40.7,31.7,28.5,39.0,18.4,29.9,47.9,88.8,65.7,87.9,83.4,4.7,37.7,19.5,34.3,25.9,9.9,4.4,6.4,62.6,14.1,11.2
세종,15.3,41.7,24.4,8.1,8.6,22.3,3.0,1.6,0.6,77.5,43.6,1.8,1.7,13.6,41.7,8.3,3.0,0.9,48.9,40.0,22.4,27.1,35.8,44.5,28.6,39.3,19.5,42.7,46.5,85.1,62.2,85.3,80.8,4.1,39.1,22.9,36.1,27.0,13.1,4.5,6.3,58.8,19.0,12.7
경기,16.6,41.5,25.9,10.7,8.5,21.8,4.0,2.4,0.8,62.3,61.5,3.0,2.0,20.6,49.6,11.2,4.7,1.2,65.0,40.2,15.3,26.8,37.1,24.3,27.5,41.0,20.8,34.6,47.0,88.0,63.0,86.9,84.0,5.8,41.5,22.4,39.0,27.1,14.9,5.8,6.3,56.9,20.1,13.5


In [29]:
df_1.to_csv('personal_health_all_element_percent.csv')

### (2) 전체 세부항목들에 대한 순위

In [30]:
df_2 = df.copy()

num_col = len(df_2.columns)
multi_col_1 = [f'{j + 1}' for j in range(num_col)]
df_2.columns = multi_col_1
df_2.columns = df_2.columns.astype(int)

plus_col = [1,2,3,21,23,24,25,29,30,31,32,33,42]
plus_df = df_2[plus_col].rank(method='min') 
minus_df = df_2.loc[:, ~df_2.columns.isin(plus_col)].rank(method='min', ascending=False)
result = pd.concat([plus_df, minus_df], axis=1)
result = result.sort_index(axis=1)

multi_col_2 = [f'T{j + 1}' for j in range(num_col)]
result.columns = multi_col_2
result = result.astype(int)
result

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29,T30,T31,T32,T33,T34,T35,T36,T37,T38,T39,T40,T41,T42,T43,T44
시도,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
서울,12,13,4,14,12,2,16,13,15,15,14,12,12,16,9,3,16,12,13,5,16,1,9,1,15,7,7,7,7,14,13,14,16,2,4,5,2,2,4,5,17,5,3,3
부산,9,10,13,12,3,9,14,13,6,12,8,15,16,15,4,2,14,14,6,15,15,15,13,5,12,11,5,12,16,17,15,15,17,15,14,17,17,17,17,14,14,16,16,16
대구,4,6,8,10,6,17,13,10,10,9,7,15,11,14,13,1,13,12,9,13,14,7,17,2,4,17,17,9,14,15,16,15,15,1,6,11,13,14,13,8,14,12,12,17
인천,14,3,6,10,2,8,8,7,3,17,5,9,12,7,9,14,10,8,3,1,3,5,9,3,5,4,2,2,10,10,11,12,13,16,11,6,7,7,6,9,7,7,8,7
광주,3,8,13,17,11,11,4,3,5,6,3,3,9,8,1,12,7,2,7,4,9,4,4,8,15,2,9,6,1,2,3,3,3,8,8,2,5,12,5,7,12,3,9,3
대전,11,4,4,14,8,14,11,10,6,5,10,12,12,13,7,17,9,3,4,9,12,3,4,7,7,12,3,15,8,7,8,8,11,8,2,1,4,5,8,3,16,1,6,9
울산,15,17,17,13,4,7,9,15,10,4,4,6,2,11,3,13,11,3,8,16,13,8,16,10,13,15,13,16,15,16,16,15,12,14,7,15,15,8,1,14,2,17,16,14
세종,2,12,3,1,6,4,17,17,17,2,17,17,16,17,17,4,17,17,17,13,17,6,2,17,14,13,11,1,11,4,10,8,10,17,3,3,11,4,9,12,7,10,3,6
경기,6,9,12,16,8,9,12,10,15,14,12,11,10,12,2,15,12,14,10,11,10,8,6,4,10,7,5,8,13,13,12,13,14,3,1,7,3,3,3,4,7,6,1,2


In [31]:
result.to_csv('personal_health_all_element_rank.csv')

### (3) 세부항목 합산 순위 (신체활동, 비만, 흡연, ... )
    - T1 : 신체활동
    - T2 : 비만
    - T3 : 흡연
    - T4 : 음주
    - T5 : 식생활
    - T6 : 구강보건
    - T7 : 개인위생
    - T8 : 아토피/천식
    - T9 : 스트레스 인지율
    - T10 : 우울감 경험률
    - T11 : 자살 생각률 & 자살 계획률
    - T13 : 주중 평균 수면시간
    - T14 : 주관적 행복감 경험률
    - T15 : 외로움 경험률
    - T16 : 중등도 이상 범불안장애 경험률

In [59]:
res = give_rank_to_df_2( include_multiCol_df_list, plus_col_list )
suicide = include_multiCol_df_list[8][[('T3', '분율'), ('T4', '분율')]].astype(float).mean(axis=1).rank(method='min', ascending=False).astype(int)
res['T11'] = suicide
res.drop(columns=['T12'], inplace=True)
res.columns = ['T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15',]
res

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15
시도,,,,,,,,,,,,,,,
서울,11,9,16,14,7,5,13,1,2,2,4,17,5,3,3
부산,12,6,14,12,17,9,17,17,17,17,16,14,16,16,16
대구,4,14,13,9,14,11,16,4,13,14,10,14,12,12,17
인천,7,3,9,9,2,1,12,10,7,7,7,7,7,8,7
광주,9,17,2,5,3,6,1,4,5,12,6,12,3,9,3
대전,5,16,10,7,6,7,9,1,4,5,5,16,1,6,9
울산,17,6,7,9,16,17,15,13,15,8,17,2,17,16,14
세종,2,1,17,16,11,13,10,7,11,4,9,7,10,3,6
경기,10,14,12,15,9,3,14,1,3,3,3,7,6,1,2


In [60]:
res.to_csv('personal_health_middle_classification_rank.csv')